Functions:
- Load/Extract new file
-  

In [46]:
import pandas as pd
import os
from datetime import datetime

In [47]:
master_file_directory = 'landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx'

processes_files_storage_path ='C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/ready_to_upload'

<>:1: SyntaxWarning: invalid escape sequence '\H'
<>:1: SyntaxWarning: invalid escape sequence '\H'
C:\Users\paxm\AppData\Local\Temp\ipykernel_37348\2908687198.py:1: SyntaxWarning: invalid escape sequence '\H'
  master_file_directory = 'landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx'


In [48]:






# masterfile = pd.read_excel( f'..\landing_zone\2026\2025-06\Honda\2026_Honda_Accessories-06-2025.xlsx', engine='openpyxl')

masterfile = pd.read_excel(f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/2026_Honda_Accessories-06-2025.xlsx', engine='openpyxl')





In [49]:
master_file_path =f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/2026-Honda_Accessories-06_2025.xlsx'

In [50]:

# create a function that will read take in an excel file as an input, and find all the worskheets in the file and create a dataframe for each worksheet. This function will return a dictionary of dataframes with the worksheet names as the keys.
def read_excel_sheets(file_path):
    """
    Reads an Excel file and returns a dictionary of DataFrames for each sheet.
    
    :param file_path: Path to the Excel file.
    :return: Dictionary with sheet names as keys and DataFrames as values.
    """
    xls = pd.ExcelFile(file_path)
    sheets_dict = {sheet_name: pd.read_excel(xls, sheet_name=sheet_name, engine='openpyxl') for sheet_name in xls.sheet_names}

    # Optionally, you can close the Excel file after reading
    # first check if the file is open and the sheet_dict is not empty
    if sheets_dict and hasattr(xls, 'close'):
        xls.close()
    return sheets_dict, xls.sheet_names
 

In [51]:
# create a function to clean up the dataframes by, looping through each dataframe in the dictionary, check every column name and replace any \n with a space from the names. and make sure that if the column name has a space at the end, it is removed. and ensure that their no 2 spaces in the column names.    


def clean_dataframe_columns(df_dict):
    """
    Cleans the column names of DataFrames in a dictionary by replacing newline characters with spaces.
    
    :param df_dict: Dictionary of DataFrames.
    :return: Dictionary with cleaned DataFrames.
    """
    for key, df in df_dict.items():
        df.columns = [col.replace('\n', ' ') for col in df.columns]
        df.columns = [col.replace('  ', ' ') for col in df.columns]
        df.columns = [col.strip() for col in df.columns]
        df_dict[key] = df
    return df_dict
  

In [123]:

common_columns = ["Description", "Part Number", "List Price", "FRT", "Installed Price", "Comments"]


In [91]:
# create a function that will take a dic of dataframes, loop through the dataframes and will create other dataframes based on the following instructions: Find the common columns which are : Description, Part, Cost, Hours, Price, and Comments.. for all the remaining columns which are unique model columns, create a new dataframe with the names as the column name. The dataframes will be made of a unique model column and all the common columns. The new dataframe will be named as the column name of the unique model column. The function will return a dictionary of dataframes with the unique model column names as the keys and the new dataframes as the values. For each unique model to determine whether a row will be added to the df, check in the master df if the column is not null and the value is not empty. If the value is not null and not empty, then add the row to the new dataframe. If the value is null or empty, then do not add the row to the new dataframe. The function will also check if the column name is in the master df columns before creating the new dataframe. Also, for the value to be considered to be not empty, it should have a "•" in the cell. If the value does not have a "•", then it is considered empty and will not be added to the new dataframe. 

def create_unique_trim_dfs_old(master_df_dict):
    """
    Creates DataFrames for each unique model column in the master DataFrame.
    
    :param master_df_dict: Dictionary of DataFrames with sheet names as keys.
    :return: Dictionary of DataFrames with unique model column names as keys.
    """
    unique_model_dfs_dict = {}

    for model in master_df_dict.keys():
        master_df = master_df_dict.get(model)
        
        if master_df is not None:

            # Loop through each column in the master DataFrame
            for column in master_df.columns:
                if column not in common_columns and pd.notnull(master_df[column]).any():
                        
                    # # Create a new DataFrame for the unique model column
                    new_df = master_df[[column] + common_columns].copy()
                    print(new_df.head(1))                    
                    # # Filter rows where the unique model column has a "•" and is not null or empty
                    
                    # # re-write the newdf population to check if the cell is empty or contains string "N/A", if yes, then do not add the data. otherwise, add the data 
                    # new_df = new_df[new_df[column].notnull() & (new_df[column] != '')] 
                
                    # # If the new DataFrame is not empty, add it to the dictionary
                    # if not new_df.empty:
                    #     unique_model_dfs_dict[column] = new_df
        
    return unique_model_dfs_dict


In [118]:
def create_unique_trim_dfs(master_df_dict):
    """
    Creates DataFrames for each unique model column in the master DataFrame.
    
    :param master_df_dict: Dictionary of DataFrames with sheet names as keys and master sheet def/content as value.
    :return: Dictionary of DataFrames with unique trims column names as keys and unique trim df as value.
    """
    unique_model_dfs_dict = {}

    # Loop through all the keys of the dict
    # if the value is not empty, then break it down
#     print(master_df_dict)


    for model in master_df_dict.keys():
        

        master_df = master_df_dict.get(model)
        
        if master_df is not None:
            
            # Loop through each column in the master DataFrame
            for column in master_df.columns:
                if column not in common_columns and pd.notnull(master_df[column]).any():
                    
                    # # Create a new DataFrame for the unique model column
                    new_df = master_df[[column] + common_columns].copy()
                    
                # Filter rows where the unique model column has a "•" and is not null or empty                    
                # re-write the newdf population to check if the cell is empty or contains string "N/A", if yes, then do not add the data. otherwise, add the data 
                    new_df = new_df[new_df[column].notnull() & (new_df[column] != '')] 
                
                    # If the new DataFrame is not empty, add it to the dictionary
                    if not new_df.empty:
                        unique_model_dfs_dict[model][column] = new_df
    # unique_model_dfs_dict = master_df_dict  
    return unique_model_dfs_dict 

In [54]:
# Create a function that takes in a dictionary of dataframes and a string for the year and adds a column "Year" on each dataframe and add in the year value to the column. The function will return the dictionary of dataframes with the new column added. I want to add the new column year at the beginning of the dataframe. The year will be a string value, for example, "2026". The function will loop through each dataframe in the dictionary and add the column to each dataframe.

def add_year_column(df_dict, year):
    """
    Adds a 'Year' column to each DataFrame in the dictionary.
    
    :param df_dict: Dictionary of DataFrames.
    :param year: Year to be added as a string.
    :return: Dictionary with DataFrames having the 'Year' column added.
    """
    for key, df in df_dict.items():
        df.insert(0, 'Year', year)  # Insert 'Year' column at the beginning
        df_dict[key] = df
    return df_dict



# create a function that removed the year column from the dataframes in the dictionary. The function will loop through each dataframe in the dictionary and remove the column "Year" if it exists. The function will return the dictionary of dataframes with the column removed.
def remove_year_column(df_dict):
    """
    Removes the 'Year' column from each DataFrame in the dictionary if it exists.
    
    :param df_dict: Dictionary of DataFrames.
    :return: Dictionary with DataFrames having the 'Year' column removed.
    """
    for key, df in df_dict.items():
        if 'Year' in df.columns:
            df.drop(columns=['Year'], inplace=True)
        df_dict[key] = df
    return df_dict


In [55]:
# create a function that takes in a dictionary of dataframes and saves it as an excel sheet with each dataframe as a separate sheet. The function will take in the dictionary of dataframes and a file path to save the excel file. The function will return nothing. The function will use the pandas ExcelWriter to write the dataframes to the excel file. The function will also check if the file path exists, if not, it will create the directory. The function will also check if the file already exists, if yes, it will overwrite the file. I want to modify this function, the input will be a dictionary of a dataframes, a directory path, the year, and make. Withe provided directory path, th goal is to find the country folder first, say 2026, if it doesn't exist create it, and then find the make, it doesn't exist create it based on the provided string, and then dave the fail in the format of "YYYY-Make_Accessories-UploadFile-timestamp with seconds.xlsx". The function will also ensure that the directory exists before saving the file. 

def save_dfs_to_excel(df_dict, directory_path, year, make, trim):
    """
    Saves a dictionary of DataFrames to an Excel file with each DataFrame as a separate sheet.
    
    :param df_dict: Dictionary of DataFrames.
    :param directory_path: Directory path to save the Excel file.
    :param year: Year to be included in the file name.
    :param make: Make to be included in the file name.
    :return: None
    """


    # Create the directory structure if it doesn't exist
    year_dir = os.path.join(directory_path, str(year))
    make_dir = os.path.join(year_dir, make)
    
    os.makedirs(make_dir, exist_ok=True)

    # Create the file name with timestamp
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    file_name = f"{year}-{make}-{trim}_Accessories-UploadFile-{timestamp}.xlsx"
    file_path = os.path.join(make_dir, file_name)

    # Save the DataFrames to an Excel file
    with pd.ExcelWriter(file_path, engine='openpyxl') as writer:
        for sheet_name, df in df_dict.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    
    print(f"DataFrames saved to {file_path}")

In [56]:


# save_dfs_to_excel(unique_model_dfs_dict, processes_files_storage_path, year='2026', make='Honda', trim='CRV')



In [57]:
# write a function that takes in a dictionary of dataframes, a string with manufacture name, and a string with the year and runs a database search to a database to get a model string of the vehicle. If the model value is not empty, then create a new column names Model and populate the model number into the whole column. 


def add_model_column(df_dict, make, year):
    """
    Adds a 'Model' column to each DataFrame in the dictionary based on a database search.
    
    :param df_dict: Dictionary of DataFrames.
    :param make: Make of the vehicle.
    :param year: Year of the vehicle.
    :return: Dictionary with DataFrames having the 'Model' column added.
    """
    # Placeholder for database search logic
    # For demonstration, we'll assume a simple mapping
    model_mapping = {
        'Honda': {'2026': 'CRV', '2025': 'Civic'},
        'Toyota': {'2026': 'RAV4', '2025': 'Camry'}
    }
    
    model = model_mapping.get(make, {}).get(year, '')

    for key, df in df_dict.items():
        if model:
            df['Model'] = model
        else:
            df['Model'] = ''  # or handle as needed
        df_dict[key] = df
    
    return df_dict

In [58]:
# create a utility that loads a table of vehicle profiles into a csv. This utility will do regular checks to see if there are any new vehicles added to the table and if so, it will update the csv file with the new vehicles. TO make updates, after getting the response from the database, it will check if there is any difference in teh already existing values, in csv catalogue and will update the csv accordingly. Let's say a particular record has had some changes, we'll then do it that way. Make sure that all data are in lower case to make sure that there are no duplicates because of case sensitivity. I want you to create a function, and do not worry about the query, prented as if the query has been emplimentated, and a path where to store the csv will be provided, and this where it will always be.  

def update_vehicle_catalogue(csv_path, new_data):
    """
    Updates the vehicle catalogue CSV with new data from the database.
    
    :param csv_path: Path to the vehicle catalogue CSV file.
    :param new_data: New data from the database as a DataFrame.
    :return: None
    """
    # Load existing catalogue if it exists
    if os.path.exists(csv_path):
        existing_catalogue = pd.read_csv(csv_path)
        existing_catalogue = existing_catalogue.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    else:
        existing_catalogue = pd.DataFrame()

    # Convert new data to lower case for consistency
    new_data = new_data.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    # Identify new or updated records
    if not existing_catalogue.empty:
        combined = pd.concat([existing_catalogue, new_data]).drop_duplicates(keep=False)
    else:
        combined = new_data

    # If there are changes, update the CSV
    if not combined.empty:
        updated_catalogue = pd.concat([existing_catalogue, combined]).drop_duplicates().reset_index(drop=True)
        updated_catalogue.to_csv(csv_path, index=False)
        print(f"Vehicle catalogue updated at {csv_path}")
    else:
        print("No updates to the vehicle catalogue.")

In [59]:
# Placeholder for database search logic

model_mapping = {
        'Honda': {'2026': 
                  {'CRV': 
                    {'LX': '2026-Honda-LX', 'Sport': '2026-Honda-Sport', 'Trailsport': '2026-Honda-Trailsport', 'EX-L': '2026-Honda-EX-L', 'Touring': '2026-Honda-Touring', 'Sport-Hybrid':'2026-Honda-Sport-Hybrid'}
                    }, 
                   'Civic': {'LX': '2026-Honda-LX', 'Sport': '2026-Honda-Sport'},
        'Toyota': {'2026': {'RAV4':{'RAV4': '2026-Toyota-RAV4'}, 'Camry': {'Camry': '2026-Toyota-Camry'}}}
    }}  


In [60]:
# create a function that takes in a make, year, and model name and returns a vehicle model number. The function will check if the model name is in the model_mapping dictionary, if yes, it will return the model number, if not, it will return an empty string.

def get_vehicle_model_number(make, year, trim, model_name):
    """
    Returns the vehicle model number based on make, year, and model name.
    
    :param make: Make of the vehicle.
    :param year: Year of the vehicle.
    :param model_name: Name of the vehicle model.
    :param trim: Trim of the vehicle model.
    :return: Vehicle model number or empty string if not found.
    """
    return model_mapping.get(make, {}).get(year, {}).get(trim, {}).get(model_name, '')

In [61]:
get_vehicle_model_number(make='Honda', year='2026', trim='CRV', model_name='Sport-Hybrid')

'2026-Honda-Sport-Hybrid'

In [62]:
# write a function to add a model column to the unique_model_dfs_dict dictionary. The function will take in a make, year, and model name and will call the get_vehicle_model_number function to get the model number. If the model number is not empty, it will add a new column named Model to each dataframe in the dictionary and populate it with the model number. If the model number is empty, it will not add the column.

def add_model_column(df_dict, make, year, trim):
    """
    Adds a 'Model' column to each DataFrame in the dictionary based on a database search.
    Fetches model number using get_vehicle_model_number function.

    :param df_dict: Dictionary of DataFrames.
    :return: Dictionary with DataFrames having the 'Model' column added.
    """

      # Example trim, can be modified as needed
    
    for key, df in df_dict.items():
        model_name = key.split()[0]  # Assuming the model name is part of the key
        model_number = get_vehicle_model_number(make=make, year=year, trim=trim, model_name=model_name)
        
        if model_number:
            print("I am here")
            df.insert(1, 'Model', model_number)
            df.drop(columns=[model_name], inplace=True, errors='ignore')  # Remove 'Description' if it exists
        else:
            df.insert(1, 'Model', '')  # or handle as needed
        df_dict[key] = df
    
    return df_dict

In [63]:
# create a drop model column function that will remove the model column from the dataframes in the dictionary. The function will loop through each dataframe in the dictionary and remove the column "Model" if it exists. The function will return the dictionary of dataframes with the column removed.

def drop_model_column(df_dict): 
    """
    Removes the 'Model' column from each DataFrame in the dictionary if it exists.
    
    :param df_dict: Dictionary of DataFrames.
    :return: Dictionary with DataFrames having the 'Model' column removed.
    """
    for key, df in df_dict.items():
        if 'Model' in df.columns:
            df.drop(columns=['Model'], inplace=True)
        df_dict[key] = df
    return df_dict

In [136]:
def combine_sheets(df_dict):
    """
    Combines all DataFrames in the dictionary into a single DataFrame.
    
    :param df_dict: Dictionary of DataFrames.
    :return: Combined DataFrame.
    """
    combined_df = pd.concat(df_dict.values(), ignore_index=True)

    print(f'Combined_df_columns: {combined_df['Model'].unique()}')
    print(f'dict_keys: {df_dict.keys()}')
    return combined_df

In [133]:
# create a main function that will call all the functions in the correct order to process the master file and save the dataframes to an excel file. The function will take in the master file path, processes files storage path, year, and make as parameters. The function will return nothing.

def main(master_file_path, processes_files_storage_path, year, make, trim):

    # Step 1: Read the master file and create a dictionary of DataFrames
    master_df_dict, sheet_names = read_excel_sheets(master_file_path)
    print(f"Master file read successfully with {len(master_df_dict)} sheets.")
    
    # Step 2: Clean the DataFrame columns
    master_df_dict = clean_dataframe_columns(master_df_dict)


    # for model_master_file in master_df_dict.keys():
        
    # Step 3: Create unique model DataFrames

    unique_model_dfs_dict = create_unique_trim_dfs(master_df_dict)
    print (f"Unique model DataFrames created with {len(unique_model_dfs_dict)} models.")

    # # Step 4: Add year column to unique model DataFrames
    # unique_model_dfs_dict = add_year_column(unique_model_dfs_dict, year)
    # print(f"Year column added to unique model DataFrames for year {year}.")
    
    # # Step 5: Add model column to unique model DataFrames
    # unique_model_dfs_dict = add_model_column(df_dict= unique_model_dfs_dict, make= make, year=year, trim=trim)
    # # print(f"Model column added to unique model DataFrames for make {make} and year {year}.")

    # combined_dataframes = combine_sheets(unique_model_dfs_dict)
    # unique_model_dfs_dict[f'{make}_{trim}_combined_sheets'] = combined_dataframes

    # # Step 6: Save the DataFrames to an Excel file
    # save_dfs_to_excel(unique_model_dfs_dict, processes_files_storage_path, year, make, trim)
    
    
    # unique_model_dfs_dict = master_df_dict

    return unique_model_dfs_dict

In [132]:
file_name ='2025-Honda-Master-June25'

master_file_path = f'C:/Users/paxm/OneDrive - PBS SYSTEMS/Desktop/Office/Projects/OEM Accessory project/OEM Accessories_v1/landing_zone/2026/2025-06/Honda/{file_name}.xlsx'

# print(master_file_path)
extract_df = main(master_file_path, processes_files_storage_path, year='2026', make='Honda',trim='CRV')


Master file read successfully with 8 sheets.


KeyError: 'CivicHatch-Master'

In [107]:
extract_df.keys()

dict_keys(['CivicHatch-Master', 'HRV-Master', 'Pilot-Master', 'Passport-Master', 'ODYSSEY-Master', 'Ridgeline-Master', 'CRV-Master', 'Accord-Master'])

In [106]:
extract_df.get('Passport-Master')

,Sport,TrailSport,Black Edition,Description,Part Number,List Price,FRT,Installed Price,Comments
0,•,•,•,Protection Package,24PAS-PRO-PK1 or\n24PAS-PRO-PK2,$529.00,0.2,$561.40,24PAS-PRO-PK1 applies to Sport and Touring tri...
1,•,•,•,Protection Package - Advance,24PAS-PRO-PKA1 or\n24PAS-PRO-PKA2,$882.00,1.4,"$1,108.80",24PAS-PRO-PKA1 applies to Sport and Touring tr...
2,•,•,•,Utility Package,22PAS-UTL-PK1,"$2,985.00",1.9,"$3,292.80",Inluding:\nCrossbars\nFender Flares\nRunning B...
3,•,•,•,Cold Weather Package,19PAS-COLD-PK,$374.30,1.1,$552.50,5% Discount Applied. \n\nInclude: \nEngine Blo...
4,•,•,NaN,"Tow Package, 3500 lbs",22PAS-TOW-1A or\n22PAS-TOW-1B,"$1,431.65",1.2,"$1,626.05",5% Discount Applied.\n\n22PAS-TOW-1A includes ...
...,...,...,...,...,...,...,...,...,...
81,NaN,•,NaN,Diffused Sky Pearl (B-638P),TUCH-B638P-A1,$21.50,0.0,$21.50,NaN
82,NaN,•,•,Radiant Red Metallic II (R-580M),TUCHR580MA1,$21.50,0.0,$21.50,NaN
83,•,•,•,Fine Sanding Needles,TUCH-NEEDLES,$15.75,0.0,$15.75,Quantity of 5.
84,•,•,•,Fine Sanding Sticks,TUCH-STICKS,$4.00,0.0,$4.00,Quantity of 5.
